# Find Stations

First you have to import the meteomatics module and the datetime module

In [27]:
import datetime as dt
import meteomatics.api as api

from __future__ import print_function

In [28]:
# Opening JSON file with my credentials
import json

creds = open('mycredentials.json')
creds = json.load(creds)
username = creds["username"]
password = creds["password"]

In [29]:
###Credentials:
username = 'python-community'
password = 'Umivipawe179'

Input here a startdate, an enddate and the time interval, all as datetime-objects. The interval tells you, if you get the data in hourly steps, daily steps or every five minutes in between the startdate and the enddate. Make sure that the enddate is not in the future, because there are no station measurements from the future.

In [30]:
startdate_station_ts = dt.datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0)-dt.timedelta(days=2)
enddate_station_ts = startdate_station_ts + dt.timedelta(days=1)
interval_station_ts = dt.timedelta(hours=1)

Choose the parameters you want to get and put them into a list. Check here which parameters are available: https://www.meteomatics.com/en/api/available-parameters/

In [31]:
parameters_station_ts = ['t_2m:C']

Optional: Input here an elevation (height above sea level). It will look first for stations that are close to this height.

In [32]:
elevation = None

Optional: Define here a location, from where you are looking for a station. 
There are different possibilities to define the location in this query:
+ location = “lat,lon” ex: location = ‘47,8’
+ location = “lat_max,lon_min_lat_min,lon_max” ex: location = ‘47,8_40,15’ (This covers an area from 40°N to 47°N and from 8°E to 15°E)
+ location = ”predefined area” ex: location = ‘uk’

Pay attention: In contrary to other queries, the coordinates have to be in the form of strings without spaces in between.

In [38]:
location = '47.21088047509185,8.999690583582241_46.90356866152617,9.611075278937145'

In the following, the request will start. If there is an error in the request as for example a wrong parameter or a date that doesn't exist, you get a message.

In [39]:
print("find stations:")
try:
    met = api.query_station_list(username, password, startdate=startdate_station_ts, enddate=enddate_station_ts,
                                 parameters=parameters_station_ts, location=location, elevation=elevation)
    print(met.head())
except Exception as e:
    print("Failed, the exception is {}".format(e))

find stations:
  Station Category Station Type     ID Hash  WMO ID Alternative IDs  \
0            SYNOP         SYNO  1177494710   69900             VAD   
1            SYNOP         SYNO  2880089697   66850             GLA   
2            SYNOP         SYNO  4088031572   66860             RAG   
3            SYNOP         SYNO  4103067262   66820             ELM   
4            SYNOP         SYNA  1932528201   66850             GLA   

        Name Elevation            Start Date              End Date  \
0      Vaduz      457m  2017-02-28T23:00:00Z  2022-11-10T18:00:00Z   
1     Glarus      517m  2017-02-28T23:00:00Z  2022-11-10T18:00:00Z   
2  Bad Ragaz      497m  2017-02-28T23:00:00Z  2022-11-10T18:00:00Z   
3        Elm      958m  2017-02-28T23:00:00Z  2022-11-10T18:00:00Z   
4     Glarus      517m  2017-01-01T00:00:00Z  2022-11-10T17:00:00Z   

   Horizontal Distance  Vertical Distance  Effective Distance      lat  \
0                 -999               -999                -999  

a:\Miniconda\envs\meteomatics\lib\site-packages\meteomatics\api.py:163: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  sl.drop('Location Lat,Lon', 1, inplace=True)


As output you get a pandas dataframe filled with stations. The best matching stations to your request appear on top.

![09_find_stations_output](api_demo_images/09_find_stations_output.png)

Now you can work on the data by using pandas commands. Here are some examples how you can access to the different datapoints.

In [40]:
met

,Station Category,Station Type,ID Hash,WMO ID,Alternative IDs,Name,Elevation,Start Date,End Date,Horizontal Distance,Vertical Distance,Effective Distance,lat,lon
0,SYNOP,SYNO,1177494710,69900,VAD,Vaduz,457m,2017-02-28T23:00:00Z,2022-11-10T18:00:00Z,-999,-999,-999,47.1275,9.51758
1,SYNOP,SYNO,2880089697,66850,GLA,Glarus,517m,2017-02-28T23:00:00Z,2022-11-10T18:00:00Z,-999,-999,-999,47.0346,9.06696
2,SYNOP,SYNO,4088031572,66860,RAG,Bad Ragaz,497m,2017-02-28T23:00:00Z,2022-11-10T18:00:00Z,-999,-999,-999,47.0166,9.50259
3,SYNOP,SYNO,4103067262,66820,ELM,Elm,958m,2017-02-28T23:00:00Z,2022-11-10T18:00:00Z,-999,-999,-999,46.9237,9.17535
4,SYNOP,SYNA,1932528201,66850,GLA,Glarus,517m,2017-01-01T00:00:00Z,2022-11-10T17:00:00Z,-999,-999,-999,47.0346,9.06696
5,SYNOP,SYNO,2030955241,69900,VAD,Vaduz,457m,2017-01-01T00:00:00Z,2022-11-10T17:00:00Z,-999,-999,-999,47.1275,9.51758


In [35]:
best_fitting_station = met.iloc[0]
name_of_best_fitting_station = met.loc[0, 'Name']
station_with_least_vertical_distance = met.iloc[abs(met['Vertical Distance']).idxmin()]